In [37]:
from faker import Faker
import pandas as pd
import numpy as np
import random
from sqlalchemy import create_engine ,text
import os
from sqlalchemy import text


In [13]:
# 📊 Générateur de données simulées fiscales
fake = Faker()

def generer_donnees_fiscales(n=10000, anomalie_rate=0.1):
    data = []
    for _ in range(n):
        revenu = random.randint(20000, 200000)
        revenu_declare = revenu * random.uniform(0.3, 0.8) if random.random() < anomalie_rate else revenu
        impot = round(revenu_declare * 0.25, 2)
        depenses = random.randint(5000, 70000)
        charges = round(depenses * random.uniform(0.1, 0.4), 2)
        data.append({
            'id_contribuable': fake.uuid4(),
            'nom': fake.last_name(),
            'prenom': fake.first_name(),
            'region': fake.city(),
            'revenu_annuel': revenu,
            'revenu_declare': revenu_declare,
            'impot_declare': impot,
            'depenses': depenses,
            'charges': charges,
            'type_revenu': random.choice(['salaire', 'indépendant', 'patrimoine']),
        })
    return pd.DataFrame(data)

# Générer un DataFrame
df_fisc = generer_donnees_fiscales(n=10000)



In [16]:
df_fisc.shape

(10000, 10)

In [19]:

# 📁 Chemin absolu du dossier
chemin_dossier = r"C:\Users\POATAN\Desktop\détection d’anomalies dans les données d’impôts\data"

# 📁 Créer le dossier s'il n'existe pas
os.makedirs(chemin_dossier, exist_ok=True)

# 💾 Chemin complet du fichier CSV
fichier_csv = os.path.join(chemin_dossier, "donnees_fiscales.csv")

# 📝 Enregistrement du DataFrame
df_fisc.to_csv(fichier_csv, index=False)

print(f"[✅] Données fiscales sauvegardées dans : {fichier_csv}")


[✅] Données fiscales sauvegardées dans : C:\Users\POATAN\Desktop\détection d’anomalies dans les données d’impôts\data\donnees_fiscales.csv


In [38]:
# 📡 Connexion PostgreSQL (via SQLAlchemy)
DB_USER = 'postgres'
DB_PASSWORD = 'password'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'fiscal_db'

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

print("[✅] Connexion PostgreSQL établie avec SQLAlchemy")


[✅] Connexion PostgreSQL établie avec SQLAlchemy


In [25]:
# 📥 Charger les données dans PostgreSQL
df_loaded = pd.read_csv(r"C:\Users\POATAN\Desktop\détection d’anomalies dans les données d’impôts\data\donnees_fiscales.csv")

df_loaded.to_sql(
    name='donnees_fiscales_raw',
    con=engine,
    if_exists='replace',
    index=False
)

print("[✅] Table 'donnees_fiscales_raw' créée et données insérées dans PostgreSQL")


[✅] Table 'donnees_fiscales_raw' créée et données insérées dans PostgreSQL


In [32]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM donnees_fiscales_raw LIMIT 5"))
    for row in result:
        print(row)


('6198731a-b039-4f01-8a14-1155b40e2184', 'Robinson', 'Amanda', 'Nicoleton', 196550, 196550.0, 49137.5, 50348, 7046.79, 'indépendant')
('c0ec020e-96d8-45e6-977a-b4ea07bbc047', 'Andrews', 'Kenneth', 'East Adrian', 170746, 170746.0, 42686.5, 35501, 8175.9, 'salaire')
('33f70698-6c1d-4fa7-a59b-523cc51a1cce', 'Phillips', 'Christina', 'Port Jeanetteburgh', 91071, 91071.0, 22767.75, 22864, 3687.22, 'patrimoine')
('31c21c2d-5530-4ac9-b145-4c659c0bb82f', 'Wise', 'Francisco', 'Perrybury', 85316, 85316.0, 21329.0, 25892, 6476.58, 'salaire')
('e2cac734-f593-4f41-9c50-a81400968d71', 'Clayton', 'Laura', 'Crystalbury', 196606, 196606.0, 49151.5, 10272, 3351.55, 'salaire')


In [ ]:
from sklearn.ensemble import IsolationForest

# Charger les données depuis la vue DBT
query = "SELECT * FROM stg_donnees_fiscales"
df_stg = pd.read_sql(query, con=engine)

# Sélectionner colonnes numériques pour ML
features = ['revenu_annuel', 'revenu_declare', 'impot_declare', 'depenses', 'charges']
X = df_stg[features]

# Entraîner Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
df_stg['anomaly_iso'] = iso_forest.fit_predict(X)

# Anomalies détectées = -1
df_stg['is_anomaly'] = df_stg['anomaly_iso'] == -1

# Résumé rapide
print(f"Anomalies détectées : {df_stg['is_anomaly'].sum()} sur {len(df_stg)}")

# Afficher quelques exemples
df_stg[df_stg['is_anomaly']].head()


Anomalies détectées : 1000 sur 10000


,id_contribuable,nom,prenom,region,revenu_annuel,revenu_declare,impot_declare,depenses,charges,type_revenu,anomaly_iso,is_anomaly
0,6198731a-b039-4f01-8a14-1155b40e2184,robinson,amanda,Nicoleton,196550,196550.000000,49137.50,50348,7046.79,indépendant,-1,True
4,e2cac734-f593-4f41-9c50-a81400968d71,clayton,laura,Crystalbury,196606,196606.000000,49151.50,10272,3351.55,salaire,-1,True
9,6743eabc-3503-4f5d-b394-de7bd3f166d6,booker,phyllis,Hamiltonland,142947,47409.447051,11852.36,17880,6138.58,indépendant,-1,True
10,4b137b64-d6d8-40bf-b442-c67219ffcb87,hunt,brandy,West Grace,193665,193665.000000,48416.25,17103,2218.28,salaire,-1,True
23,1b98e772-ebba-40da-ad97-78e776c6b206,wilson,daniel,Madisonmouth,197619,197619.000000,49404.75,50518,14455.68,patrimoine,-1,True


In [47]:
import boto3
from botocore.exceptions import NoCredentialsError

# Paramètres MinIO
MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = "minioadmin"      # par défaut, adapte si changé
MINIO_SECRET_KEY = "minioadmin"
MINIO_BUCKET = "fiscal-data"

# Sauvegarde locale parquet
parquet_file = "data/anomalies.parquet"
df_stg.to_parquet(parquet_file, index=False)
print(f"[✅] Fichier parquet sauvegardé localement : {parquet_file}")

# Connexion S3 compatible MinIO
s3_client = boto3.client(
    's3',
    endpoint_url=f"http://{MINIO_ENDPOINT}",
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    region_name='us-east-1',  # région par défaut, pas utilisée par MinIO
)

# Vérifier que le bucket existe, sinon le créer
buckets = [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]
if MINIO_BUCKET not in buckets:
    s3_client.create_bucket(Bucket=MINIO_BUCKET)
    print(f"[✅] Bucket '{MINIO_BUCKET}' créé sur MinIO")

# Upload fichier parquet
try:
    s3_client.upload_file(parquet_file, MINIO_BUCKET, "anomalies.parquet")
    print(f"[✅] Fichier '{parquet_file}' uploadé sur MinIO dans le bucket '{MINIO_BUCKET}'")
except NoCredentialsError:
    print("Erreur : Credentials MinIO invalides ou manquants")


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [46]:
!pip install boto3

  Using cached boto3-1.39.17-py3-none-any.whl (139 kB)
  Using cached botocore-1.39.17-py3-none-any.whl (13.9 MB)
  Using cached s3transfer-0.13.1-py3-none-any.whl (85 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
